In [67]:
import numpy as np
import pickle
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold



In [68]:

scaler = StandardScaler()
with open("dataset_bin.pkl", "rb") as f:
    df=pickle.load(f)

In [89]:
X = df.drop(['label'], axis=1)

y = df['label']

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [91]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [94]:
from scipy.stats import uniform
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
pipeline = make_pipeline(svm.SVC())
param_distributions = {
    'svc__C': uniform(0.1, 100),       # Samples C between 0.1 and 100
    'svc__gamma': uniform(0.001, 1),   # Samples gamma between 0.001 and 1
    'svc__kernel': ['rbf'],'svc__class_weight':['balanced', None]
}

search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,             # Try 50 different combinations
    scoring='accuracy',
    cv=5,
    random_state=70,
    verbose=4,
    n_jobs=-1
)
search.fit(X_train, y_train)
print("Best Params:", search.best_params_)
print("Best CV Score:", search.best_score_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Params: {'svc__C': np.float64(13.86338786942478), 'svc__class_weight': None, 'svc__gamma': np.float64(0.1857480115333836), 'svc__kernel': 'rbf'}
Best CV Score: 0.9250876776924111


In [78]:
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)


In [79]:
print('Accuracy:',metrics.accuracy_score(y_test, y_pred))
print('Precision:',metrics.precision_score(y_test, y_pred))
print('Recall:',metrics.recall_score(y_test, y_pred))

Accuracy: 0.9227359088030399
Precision: 0.841726618705036
Recall: 0.8624078624078624
